In [1]:
# imports for notebook boilerplate
!pip install -Uqq fastbook
import fastbook
from fastbook import *
from fastai.tabular.all import *

In [2]:
# set up the notebook for fast.ai
fastbook.setup_book()

In [5]:
! pip install pandas_datareader
import numpy as np
import pandas as pd
import os
import yaml
# For reading stock data from yahoo
from pandas_datareader.data import DataReader

# For time stamps
from datetime import datetime


     |████████████████████████████████| 107 kB 26.0 MB/s eta 0:00:01
     |████████████████████████████████| 5.4 MB 18.9 MB/s eta 0:00:01


In [11]:
# Set up start and end times for the data load - a year previous
end_time = datetime.now()
start_time = datetime(end_time.year - 1, end_time.month, end_time.day)


,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-01-31,49.180000,48.470001,49.180000,48.700001,4129300.0,47.348907
2020-02-03,48.930000,48.450001,48.619999,48.509998,1990900.0,47.164177
2020-02-04,49.090000,48.720001,48.759998,48.759998,1698800.0,47.407246
2020-02-05,49.849998,49.230000,49.459999,49.730000,2303000.0,48.350334
2020-02-06,50.099998,49.500000,49.689999,50.029999,2531000.0,48.642010


In [21]:
df = DataReader('AZN', 'yahoo', start_time, end_time)
df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2011-01-31,24.490000,24.184999,24.240000,24.450001,4079000.0,15.562676
2011-02-01,24.639999,24.440001,24.514999,24.620001,2414000.0,15.670886
2011-02-02,23.940001,23.790001,23.889999,23.855000,2114400.0,15.776698
2011-02-03,24.135000,23.879999,24.055000,23.969999,4370200.0,15.852756
2011-02-04,23.780001,23.600000,23.719999,23.780001,2330200.0,15.727097


In [22]:
df.shape

(2517, 6)

In [23]:
# check for missing values
count = df.isna().sum()
df_missing = (pd.concat([count.rename('missing_count'),
                     count.div(len(df))
                          .rename('missing_ratio')],axis = 1)
             .loc[count.ne(0)])

In [24]:
# check for missing values
df_missing

,missing_count,missing_ratio


In [25]:
dep_var = 'Close'
# define columns that are continuous / categorical
cont,cat = cont_cat_split(df, 1, dep_var=dep_var) 
print("continuous columns are: ",cont)
print("categorical columns are: ",cat)

continuous columns are:  ['High', 'Low', 'Open', 'Volume', 'Adj Close']
categorical columns are:  []


In [26]:
procs = [Normalize]
dls = TabularDataLoaders.from_df(df,procs= procs, 
                                 cat_names= cat, cont_names = cont, 
                                 y_names = dep_var, 
                                 valid_idx=list(range((df.shape[0]-50),df.shape[0])), bs=64)

In [27]:
dls.valid.show_batch()

,High,Low,Open,Volume,Adj Close,Close
0,55.419999,54.749999,55.110001,6.451500e+06,54.960000,54.959999
1,55.339999,53.959999,55.330002,4.732300e+06,53.990001,53.990002
2,54.250001,53.669998,54.250000,6.768200e+06,54.029998,54.029999
3,55.419999,54.679999,54.880001,4.306000e+06,55.300000,55.299999
4,54.709999,53.330002,54.520001,1.367060e+07,54.700001,54.700001
5,54.300000,53.520000,54.000000,8.341400e+06,53.570001,53.570000
6,53.089999,52.299999,52.340000,1.166090e+07,52.599998,52.599998
7,52.770001,51.439998,51.580002,1.478320e+07,52.610000,52.610001
8,53.419998,52.529999,53.299999,1.472520e+07,52.939997,52.939999
9,53.360000,52.599999,53.060002,1.330820e+07,52.979999,52.980000


In [32]:
# define and fit the model
learn = tabular_learner(dls, metrics=accuracy)
learn.fit_one_cycle(3)

epoch,train_loss,valid_loss,accuracy,time
0,1084.947632,1732.947144,0.000000,00:00
1,713.420837,65.922012,0.000000,00:00
2,459.654083,17.623674,0.000000,00:00


In [20]:
start_time = datetime(end_time.year - 10, end_time.month, end_time.day)

In [30]:
??TabularModel

Init signature:
TabularModel(
    emb_szs,
    n_cont,
    out_sz,
    layers,
    ps=None,
    embed_p=0.0,
    y_range=None,
    use_bn=True,
    bn_final=False,
    bn_cont=True,
    act_cls=ReLU(inplace=True),
)
Source:        
class TabularModel(Module):
    "Basic model for tabular data."
    def __init__(self, emb_szs, n_cont, out_sz, layers, ps=None, embed_p=0.,
                 y_range=None, use_bn=True, bn_final=False, bn_cont=True, act_cls=nn.ReLU(inplace=True)):
        ps = ifnone(ps, [0]*len(layers))
        if not is_listy(ps): ps = [ps]*len(layers)
        self.embeds = nn.ModuleList([Embedding(ni, nf) for ni,nf in emb_szs])
        self.emb_drop = nn.Dropout(embed_p)
        self.bn_cont = nn.BatchNorm1d(n_cont) if bn_cont else None
        n_emb = sum(e.embedding_dim for e in self.embeds)
        self.n_emb,self.n_cont = n_emb,n_cont
        sizes = [n_emb + n_cont] + layers + [out_sz]
        actns = [act_cls for _ in range(len(sizes)-2)] + [None]
        _layers = [